
# House Prices - Advanced Regression Techniques

In this notebook we will create a model to predict house prices for the **Kaggle** competition [House Prices - Advanced Regression Techniques](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques).  

The dataset used for this, is the Ames Housing dataset, but modified for learning purposes.  
And the evaluation metric used will be the Root Mean Squared Error (RMSE), on the log transformed price.  

The competition provides the following:  
- **train.csv**
> Training dataset comprised of the house's features and their sale price
- **test.csv**
> Test dataset with only the house's features but not their sale price.  
> Will act as unseen data and the final submitted prediction will be done on it.
- **data_description.txt**
> Text file describing each feature and their possible values (when categorical).


Our real objective with this project is not to create the best performing model possible, but to create different types of models and approaches, and compare their performances.  
For this, we will first do EDA on the data as well as some preprocessing and feature engineering, and then build models using the prepared data, compare them, and make a final prediction with the chosen one.

## 1. Setup

### 1.1 Imports

Before we start we need to import the resources (libraries, modules, etc.) that we will be using.  
For clarity, all imports used in this notebook will be done on this cell.  

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import root_mean_squared_error
from sklearn.feature_selection import mutual_info_regression
import joblib

# TensorFlow Decision Forests (TFDF)
import tensorflow_decision_forests as tfdf

# CatBoost 
from catboost import CatBoostRegressor, Pool

# TensorFlow
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Concatenate, Normalization, StringLookup, CategoryEncoding
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import RootMeanSquaredError

### 1.2 Global settings

We will set a fixed random seed for reproducibility.

In [ ]:
# Set random seed for reproducibility (for Python, NumPy, and TensorFlow)
tf.keras.utils.set_random_seed(33)

As the data provided has many features, we will set up **Pandas** to display all columns and rows, making it easier to explore the data.

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### 1.3 Data loading

We will load the datasets provided, **train.csv** and **test.csv**, as **Pandas** dataframes:  

In [ ]:
trainrawdata_path = '../data/raw/train.csv' # Relative path to the training dataset
traindf = pd.read_csv(trainrawdata_path)

In [ ]:
testrawdata_path = '../data/raw/test.csv' # Relative path to the test dataset
testdf = pd.read_csv(testrawdata_path)

## 2. EDA

### 2.1 Initial exploration

In [ ]:
traindf.info()

In [ ]:
testdf.info()

We can see that our training dataset has 1460 entries, with 79 features (81 columns but one is **Id** and another is the target variable **SalePrice**).  
Similarly, the test dataset provided has 1459 entries, with 79 features (80 columns couting **Id**), and as mentioned earlier, without target variable.  

Features are stored as 3 different data types: **int64**, **float64** and **object** (string).

In both datasets there seems to be missing data in some features, which we will explore and deal with later.

Let's take a quick look at a few examples of our data to see what we are dealing with:

In [ ]:
traindf.head(10)

After this first look we see that we have a mixed set of data, some columns are quantitative (numerical) like our target variable **SalePrice** or **LotArea** while many others are qualitative (categorical).  
Some of the categorical features are nominal but there are also ordinal variables like **OverallCond** which rates from 1 to 10 the overall condition of the house.

### 2.2 Target variable analysis

Our target variable is **SalePrice**, a numerical variable which shows the price at what each house got sold, in US Dollars.  
We should take a first look at its distribution:  

In [ ]:
traindf['SalePrice'].describe()

In [ ]:
# Histogram of SalePrice
plt.figure(figsize=(8, 5))
sns.histplot(traindf['SalePrice'], kde=True)
plt.title('Distribution of SalePrice')
plt.xlabel('SalePrice')
plt.ylabel('Frequency')
plt.show()
# Calculate skewness and kurtosis
print(f'Skewness of SalePrice: {traindf["SalePrice"].skew()}')
print(f'Kurtosis of SalePrice: {traindf["SalePrice"].kurt()}')
# Boxplot of SalePrice
plt.figure(figsize=(8, 1))
sns.boxplot(x=traindf['SalePrice'])
plt.title('Boxplot of SalePrice')
plt.show()

**SalePrice** has a strongly right-skewed distribution (skew = 1.88) with a heavy right tail, confirmed by the high kurtosis value (6.54) and the boxplot.  
This is expected for housing prices, where most properties are sold around a typical value, and only a few are significantly more expensive.  
The average sale price is $180,921.20, with values ranging from $34,900.00 to $755,000.00.

In [ ]:
# Histogram of log(SalePrice)
plt.figure(figsize=(8, 5))
sns.histplot(np.log1p(traindf['SalePrice']), kde=True)
plt.title('Log-Transformed SalePrice')
plt.xlabel('Log(SalePrice)')
plt.ylabel('Frequency')
plt.show()
# Calculate skewness and kurtosis
print(f'Skewness of log(SalePrice): {np.log1p(traindf["SalePrice"]).skew()}')
print(f'Kurtosis of log(SalePrice): {np.log1p(traindf["SalePrice"]).kurt()}')

The log-transformed SalePrice shows a nearly normal distribution.  
Skew = 0.12 and kurtosis = 0.81 indicate that even though not perfectly normal, the shape is close enough to be treated as such.  

We will use the log-transformed target for training to reduce the impact of errors on very cheap or very expensive houses, and to improve model stability.  
This is also required by the competition, as the evaluation is based on RMSE of the log-transformed prices.

### 2.3 Missing Values Analysis & Handling

Some of the models we will be using can deal with missing values natively, while others like those based on Neural Networks, cannot.  
For a fair comparison, we will prepare the data so that every model has the same starting point.

#### 2.3.1 Missing values on training dataset

Now lets look at which features have missing values and how many each have.

In [ ]:
traindf.isnull().sum().sort_values(ascending=False)[lambda x: x > 0]

As per the **data_description.txt** file, many of these features use NA as intented value, meaning "None".  
But there are other variables where that is not the case, like **GarageYrBlt** or **MasVnrArea**.  

As there are not that many features with missing values, lets explore them manually and deal with them accordingly.

**PoolQC** is a categorical variable which describes the quality of the pool, NA is not one of the categories but given that there is no category for "No pool", those 1453 missing values should mean that those 1453 houses have no pool.  

We can confirm this looking at how many houses have 0 pool area and checking if those entries are the same as the ones with **PoolQC** missing values.

In [ ]:
(traindf['PoolArea'] == 0).sum()

In [ ]:
traindf[(traindf['PoolArea'] == 0) & (traindf['PoolQC'].isnull())].shape

Now lets transform empty values into a string "None" to avoid issues with missing values downstream.

In [ ]:
traindf['PoolQC'] = traindf['PoolQC'].fillna('None')
traindf['PoolQC'].unique()

**MiscFeature** is also a categorical variable with NA as intended value for None, lets compare it with **MiscVal** which represents the value of said feature.

In [ ]:
(traindf['MiscVal'] == 0).sum()

In [ ]:
traindf[(traindf['MiscVal'] == 0) & (traindf['MiscFeature'].isnull())].shape

There seems to be 2 instances of **MiscVal** 0 more than the number of entries with **MiscFeature** as NA.  

In [ ]:
traindf[(traindf['MiscVal'] == 0) & (traindf['MiscFeature'].notna())]

As there are only 2 entries, we will drop them and replace the NA values of the rest of **MiscFeature** with "None".

In [ ]:
traindf = traindf.drop(index=traindf[(traindf['MiscVal'] == 0) & (traindf['MiscFeature'].notna())].index)

In [ ]:
traindf['MiscFeature'] = traindf['MiscFeature'].fillna('None')
traindf['MiscFeature'].unique()

Both **Alley** and **Fence** also use NA as None. This time there is no information to crosscheck, so we will assume all NA values are correct.

In [ ]:
traindf['Alley'] = traindf['Alley'].fillna('None')
traindf['Alley'].unique()

In [ ]:
traindf['Fence'] = traindf['Fence'].fillna('None')
traindf['Fence'].unique()

**MasVnrType** is a categorical variable that describes the type of masonry veneer, and **MasVnrArea** is a numerical variable that measures its area in square feet.  
For the type there is a None category but with None instead of NA. Lets check its values:

In [ ]:
(traindf['MasVnrArea'] == 0).sum()

In [ ]:
traindf['MasVnrArea'].isnull().sum()

In [ ]:
traindf['MasVnrType'].isnull().sum()

There seems to be some type of inconsistency here, as we have 870 missing values on **MasVnrType**, 8 missing values on **MasVnrArea**, and 859 values of 0 area.  
We should first check which rows have unexpected values.

In [ ]:
traindf[(traindf['MasVnrArea'] > 0) & (traindf['MasVnrType'].isnull())]

In [ ]:
traindf[(traindf['MasVnrArea'].isnull())]

In [ ]:
traindf[(traindf['MasVnrArea'] == 0) & ~(traindf['MasVnrType'].isnull())]

We can see 4 different cases here:  
> 8 entries where both type and area are NA  
> 2 entries where type is NA but where the area is 1.0, which would not make sense as that area value is too small  
> 3 entries where type is NA but where the area has a reasonable value  
> 2 entries where there is a valid type but the area is 0  

We are going to drop the entries with both values missing, and the two with 1.0 as area, because they are only 10 entries (<1% of the total).  

As for the other two cases, we are going to replace the missing values with the mode of the type from its neighborhood, and with the median area of the neighborhood.  

In [ ]:
traindf = traindf.drop(index=traindf[(traindf['MasVnrType'].isnull()) & (traindf['MasVnrArea'].isnull())].index)
traindf = traindf.drop(index=traindf[(traindf['MasVnrArea'] == 1.0)].index)

In [ ]:
# Create boolean mask for those rows where MasVnrType is NaN and MasVnrArea is not 0
mask1 = traindf['MasVnrType'].isna() & (traindf['MasVnrArea'] != 0)

# Create boolean mask for those rows where MasVnrType has a valid value and MasVnrArea is 0
mask2 = ~traindf['MasVnrType'].isna() & (traindf['MasVnrArea'] == 0)

# Group by Neighborhood and get the mode of MasVnrType by Neighborhood and the median of MasVnrArea.
MasVnrType_mode_Neighborhood = (traindf.groupby('Neighborhood')['MasVnrType'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else 'None'))
MasVnrArea_median_Neighborhood = traindf.groupby('Neighborhood')['MasVnrArea'].median()

# Map the mode values to the original DataFrame
traindf.loc[mask1, 'MasVnrType'] = traindf.loc[mask1, 'Neighborhood'].map(MasVnrType_mode_Neighborhood)
traindf.loc[mask2, 'MasVnrArea'] = traindf.loc[mask2, 'Neighborhood'].map(MasVnrArea_median_Neighborhood)

Lets check whether all the missing values remaining matches the number of properties without masonry veneer.

In [ ]:
print(traindf['MasVnrType'].isnull().sum())
print((traindf['MasVnrArea'] == 0).sum())
traindf[(traindf['MasVnrArea'] == 0) & ~(traindf['MasVnrType'].isnull())]

We can see there is still an entry with valid veneer type but area 0, this means that the median of that neighborhood is 0.  
As it is only one entry, our safest approach is to just drop this one entry.

In [ ]:
traindf = traindf.drop(index=traindf[(traindf['MasVnrArea'] == 0) & ~(traindf['MasVnrType'].isnull())].index)

Now we will replace the NA values in type by 'None'.

In [ ]:
traindf['MasVnrType'] = traindf['MasVnrType'].fillna('None')
traindf['MasVnrType'].unique()

**FireplaceQu** has NA as None, and the amount should match the amount of 0 **Fireplaces**.  
If so, we will just replace those NA with "None".

In [ ]:
(traindf['FireplaceQu'].isnull()).sum()

In [ ]:
(traindf['Fireplaces'] == 0).sum()

In [ ]:
traindf[(traindf['Fireplaces'] == 0) & (traindf['FireplaceQu'].isnull())].shape

In [ ]:
traindf['FireplaceQu'] = traindf['FireplaceQu'].fillna('None')
traindf['FireplaceQu'].unique()

**LotFrontage** shows the linear feet of street connected to the house.  
As there is a big number of missing values (~17%), dropping them would not be reasonable.  
Instead, we will replace those values by the median of the neighborhood.

In [ ]:
# Create boolean mask for those rows where LotFrontage is NA.
mask = traindf['LotFrontage'].isna()

# Group by Neighborhood and get the mode of LotFrontage by Neighborhood
LotFrontage_median_Neighborhood = traindf.groupby('Neighborhood')['LotFrontage'].median()

# Map the mode values to the original DataFrame
traindf.loc[mask, 'LotFrontage'] = traindf.loc[mask, 'Neighborhood'].map(LotFrontage_median_Neighborhood)

In [ ]:
(traindf['LotFrontage'].isnull()).sum()

Now we will check the garage related variables.
We have 81 missing values on **GarageQual, GarageType, GarageFinish, GarageYrBlt, GarageExposure**.  
All those are categorical and have NA as legitimate value for "no garage", except **GarageYrBlt** which is numerical (year the garage was built).  

Besides those, we can see two more variables related to the garage, **GarageArea** and **GarageCars** which are numerical variables.

Now we should check that those 81 missing values on each feature, they all match 81 unique entries, that at the same time should have all of them 0 in both Area and Cars.

In [ ]:
(traindf['GarageCars'] == 0).sum()

In [ ]:
(traindf['GarageArea'] == 0).sum()

In [ ]:
traindf[(traindf['GarageArea'] == 0) & (traindf['GarageCars'] == 0) & (traindf['GarageQual'].isnull()) 
        & (traindf['GarageType'].isnull()) & (traindf['GarageFinish'].isnull()) 
        & (traindf['GarageCond'].isnull()) & (traindf['GarageYrBlt'].isnull())].shape

After checking this we can safely replace NA values on the categorical variables with "None", and for **GarageYrBlt** we will replace with -1 as a placeholder to indicate there is no garage.

In [ ]:
for var in ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']:
    traindf[var] = traindf[var].fillna('None')
    print(f"Unique values in {var}: {traindf[var].unique()}")
traindf['GarageYrBlt'] = traindf['GarageYrBlt'].fillna(-1)
(traindf['GarageYrBlt'] == -1).sum()

As far as basement related variables, we have 11 in total:
> **BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2**, these 5 categorical features use NA as value for "no basement", and those are the ones that show missing values.  
> **BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF**, these 4 numerical features describe the area in square feet of the different sections of the basement, they have no missing values.  
> **BsmtFullBath, BsmtHalfBath**, these 2 numerical features describe the amount of full and half bathrooms that there are in the basement, they have no missing values.  

We expect to see that those entries with missing values in all 5 categorical features, should have 0 as value in all the numerical variables.  
First we will check that and replace the missing values with "None" and then we can focus in the rest of the missing values.

In [ ]:
BsmtCatCols = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']
mask = traindf[BsmtCatCols].isnull().all(axis=1)

print("Basement 1 area: ", 
      traindf[mask]["BsmtFinSF1"].value_counts())

print("Basement 2 area: ", 
      traindf[mask]["BsmtFinSF2"].value_counts())

print("Unfinished basement area: ", 
      traindf[mask]["BsmtUnfSF"].value_counts())

print("Basement total area: ", 
      traindf[mask]["TotalBsmtSF"].value_counts())

print("Basement full bathrooms: ", 
      traindf[mask]["BsmtFullBath"].value_counts())

print("Basement half bathrooms: ", 
      traindf[mask]["BsmtHalfBath"].value_counts())

With that we can safely replace in those 37 entries the missing value with "None".

In [ ]:
traindf.loc[mask, BsmtCatCols] = traindf.loc[mask, BsmtCatCols].fillna('None')

Lets check what missing values remain.

In [ ]:
traindf.isnull().sum().sort_values(ascending=False)[lambda x: x > 0]

In [ ]:
traindf[traindf["BsmtExposure"].isnull() | traindf["BsmtFinType2"].isnull()]

We can see that there is one entry with **BsmtFinType2** empty even though there is surface area and the other values make sense, and another entry where **BsmtExposure**'s value is missing even though the rest of the values indicate that there is an unfinished basement. Both cases seem to be missing information, not empty on purpose.  

Given that it is only 2 entries, we will drop them.

In [ ]:
traindf = traindf.drop(index=traindf[traindf["BsmtExposure"].isnull() | traindf["BsmtFinType2"].isnull()].index)

As for the entry with the **Electrical** feature missing, we will drop it too.

In [ ]:
traindf = traindf.drop(index=traindf[traindf["Electrical"].isnull()].index)

Before we proceed lets check all missing values have been dealt with:

In [ ]:
traindf.isnull().sum().sort_values(ascending=False)[lambda x: x > 0]

An empty list means there are no more missing values in our data.

In [ ]:
traindf.shape

We have lost in total 16 rows (~1%), but now we have a clean dataset that will not give us problems when we implement any models that cannot deal with missing values natively.

Lets save the cleaned dataset into a CSV file before proceeding with the next steps.

In [ ]:
train_clean_output_path = '../data/processed/train_clean.csv'  # Output file path
traindf.to_csv(train_clean_output_path, index=False)

#### 2.3.2 Missing values on test dataset

Now we will repeat the same operations with the test dataset provided.  
But this time we cannot drop any row, as we need to make a prediction for all entries for the **Kaggle** submission.  

We will start by loading the test set.

In [ ]:
testdf.isnull().sum().sort_values(ascending=False)[lambda x: x > 0]

We have more features with missing values in the test dataset than in the train set, but most seem to be one or two entries.  

As we cannot drop rows on the test set given that we need to make a prediction for all of them, lets first apply carefully the same transformations as with the training set and see what remains.

In [ ]:
# Replace NA with 'None' in every missing PoolQC that has PoolArea = 0
mask = (testdf["PoolArea"] == 0) & (testdf["PoolQC"].isnull())
testdf.loc[mask, "PoolQC"] = 'None'

# Replace NA with 'None' in every missing MiscFeature that has MiscVal = 0
mask = (testdf["MiscVal"] == 0) & (testdf["MiscFeature"].isnull())
testdf.loc[mask, "MiscFeature"] = 'None'

# Replace NA with 'None' in every missing Alley and Fence
testdf['Alley'] = testdf['Alley'].fillna('None')
testdf['Fence'] = testdf['Fence'].fillna('None')

# For those with both MasVnrtype and MasVnrArea missing, we will first replace the area with the median of the neighborhood from the training set
mask = testdf['MasVnrType'].isna() & (testdf['MasVnrArea'].isna())
testdf.loc[mask, 'MasVnrArea'] = testdf.loc[mask, 'Neighborhood'].map(MasVnrArea_median_Neighborhood)
# Then replace the MasVnrType with the mode of the neighborhood from the training set on those rows with a valid MasVnrArea (>0)
mask = testdf['MasVnrType'].isna() & (testdf['MasVnrArea'] > 0)
testdf.loc[mask, 'MasVnrType'] = testdf.loc[mask, 'Neighborhood'].map(MasVnrType_mode_Neighborhood)
# And for those with MasVnrArea = 0 and MasVnrType missing, we will replace the type with 'None'
mask = testdf['MasVnrType'].isna() & (testdf['MasVnrArea'] == 0)
testdf.loc[mask, "MasVnrType"] = 'None'

# Replace NA with 'None' in every missing FireplaceQu that has Fireplaces = 0
mask = (testdf["Fireplaces"] == 0) & (testdf["FireplaceQu"].isnull())
testdf.loc[mask, "FireplaceQu"] = 'None'

# Replace NA with the median LotFrontage of the neighborhood from the training set
mask = testdf['LotFrontage'].isna()
testdf.loc[mask, 'LotFrontage'] = testdf.loc[mask, 'Neighborhood'].map(LotFrontage_median_Neighborhood)

# Replace NA with 'None' in every missing categorical Garage variables, with -1 in GarageYrBlt and with 0 in GarageArea and GarageCars
# But only for those entries where all Garage variables mean there is no garage
mask = (
    ((testdf['GarageArea'].isnull()) | (testdf['GarageArea'] == 0)) &
    ((testdf['GarageCars'].isnull()) | (testdf['GarageCars'] == 0)) &
    (testdf['GarageQual'].isnull()) &
    (testdf['GarageType'].isnull()) &
    (testdf['GarageFinish'].isnull()) &
    (testdf['GarageCond'].isnull()) &
    ((testdf['GarageYrBlt'].isnull()) | (testdf['GarageYrBlt'] == 0))
)

for var in ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']:
    testdf.loc[mask, var] = 'None'
testdf.loc[mask, 'GarageYrBlt'] = -1
testdf.loc[mask, 'GarageArea'] = 0
testdf.loc[mask, 'GarageCars'] = 0



# Replace NA with 'None' in every missing categorical Basement variables, and with 0 in the numerical ones
# But only for those entries where all Basement variables mean there is no basement
mask = (
    ((testdf['BsmtFinSF1'].isnull()) | (testdf['BsmtFinSF1'] == 0)) &
    ((testdf['BsmtFinSF2'].isnull()) | (testdf['BsmtFinSF2'] == 0)) &
    ((testdf['BsmtUnfSF'].isnull()) | (testdf['BsmtUnfSF'] == 0)) &
    ((testdf['TotalBsmtSF'].isnull()) | (testdf['TotalBsmtSF'] == 0)) &
    ((testdf['BsmtFullBath'].isnull()) | (testdf['BsmtFullBath'] == 0)) &
    ((testdf['BsmtHalfBath'].isnull()) | (testdf['BsmtHalfBath'] == 0)) &
    (testdf['BsmtQual'].isnull()) &
    (testdf['BsmtCond'].isnull()) &
    (testdf['BsmtExposure'].isnull()) &
    (testdf['BsmtFinType1'].isnull()) &
    (testdf['BsmtFinType2'].isnull())
)
for var in ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']:
    testdf.loc[mask, var] = 'None'
for var in ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath']:
    testdf.loc[mask, var] = 0

Afther those transformations the missing values remaining are the following.

In [ ]:
testdf.isnull().sum().sort_values(ascending=False)[lambda x: x > 0]

In [ ]:
testdf.isnull().any(axis=1).sum()

Most missing values have been cleaned, but there are a few remnants over 22 entries that we will deal with manually.  

Using the information provided in data_description.txt we will proceed with the following:

- **MSZoning** we will replace missing values with the mode by neighborhood from the training set
- **PoolQC** we will replace missing values with the mode by neighborhood from the training set (we previously already replaced those with 0 **PoolArea** by "None")
- **Utilities** we will replace missing values with the mode by neighborhood from the training set
- **Functional** we will replace missing values with "Typ" (From documentation: Assume typical unless deductions are warranted)
- **Exterior1st** we will replace missing values with the mode by neighborhood from the training set
- **Exterior2nd** we will replace missing values with the mode by neighborhood from the training set
- **KitchenQual** we will replace missing values with the mode by neighborhood from the training set
- **MiscFeature** we will replace missing values with "Other" category (we previously already replaced those with 0 **MiscVal** by "None)
- **SaleType** we will replace missing values with the mode by neighborhood from the training set

In [ ]:
mask = testdf['MSZoning'].isna()
MSZoning_mode_Neighborhood = (traindf.groupby('Neighborhood')['MSZoning'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else 'None'))
testdf.loc[mask, 'MSZoning'] = testdf.loc[mask, 'Neighborhood'].map(MSZoning_mode_Neighborhood)

mask = testdf['PoolQC'].isna()
PoolQC_mode_Neighborhood = (traindf.groupby('Neighborhood')['PoolQC'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else 'None'))
testdf.loc[mask, 'PoolQC'] = testdf.loc[mask, 'Neighborhood'].map(PoolQC_mode_Neighborhood)

mask = testdf['Utilities'].isna()
Utilities_mode_Neighborhood = (traindf.groupby('Neighborhood')['Utilities'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else 'None'))
testdf.loc[mask, 'Utilities'] = testdf.loc[mask, 'Neighborhood'].map(Utilities_mode_Neighborhood)

mask = testdf['Functional'].isna()
testdf.loc[mask, 'Functional'] = "Typ"

mask = testdf['Exterior1st'].isna()
Exterior1st_mode_Neighborhood = (traindf.groupby('Neighborhood')['Exterior1st'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else 'None'))
testdf.loc[mask, 'Exterior1st'] = testdf.loc[mask, 'Neighborhood'].map(Exterior1st_mode_Neighborhood)

mask = testdf['Exterior2nd'].isna()
Exterior2nd_mode_Neighborhood = (traindf.groupby('Neighborhood')['Exterior2nd'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else 'None'))
testdf.loc[mask, 'Exterior2nd'] = testdf.loc[mask, 'Neighborhood'].map(Exterior2nd_mode_Neighborhood)

mask = testdf['KitchenQual'].isna()
KitchenQual_mode_Neighborhood = (traindf.groupby('Neighborhood')['KitchenQual'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else 'None'))
testdf.loc[mask, 'KitchenQual'] = testdf.loc[mask, 'Neighborhood'].map(KitchenQual_mode_Neighborhood)

mask = testdf['MiscFeature'].isna()
testdf.loc[mask, 'MiscFeature'] = "Other"

mask = testdf['SaleType'].isna()
SaleType_mode_Neighborhood = (traindf.groupby('Neighborhood')['SaleType'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else 'None'))
testdf.loc[mask, 'SaleType'] = testdf.loc[mask, 'Neighborhood'].map(SaleType_mode_Neighborhood)

All we should have left are the Basement and Garage related variables.

In [ ]:
testdf.isnull().sum().sort_values(ascending=False)[lambda x: x > 0]

In [ ]:
testdf.isnull().any(axis=1).sum()

They are only 9 entries, lets explore them:

In [ ]:
testdf[testdf.isnull().any(axis=1)]

We can see there is one entry where even though there is **GarageType** defined, the rest of the information related to the garage is missing, so we are going to assume there is no garage on the property and the type was a data error.  

The rest of the entries show enough information about basement and garage, so we will replace the missing parts using the mode by neighborhood on the categorical variables and the median on the numerical ones.

In [ ]:
# First lets deal with the row where we are assuming there is no garage
row_label = testdf[testdf["GarageCars"].isnull()].index[0]

# Now lets change the rest of the garage variables
for var in ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']:
    testdf.loc[row_label, var] = 'None'
testdf.loc[row_label, "GarageArea"] = 0.0
testdf.loc[row_label, "GarageCars"] = 0.0
testdf.loc[row_label, "GarageYrBlt"] = -1

# Now for the rest, lets replace the missing values of the categorical variables with the mode of the neighborhood from the training set
for var in ['GarageFinish', 'GarageQual', 'GarageCond', 'BsmtExposure', 'BsmtQual', 'BsmtCond']:
    mask = testdf[var].isna()
    mode = (traindf.groupby('Neighborhood')[var].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else 'None'))
    testdf.loc[mask, var] = testdf.loc[mask, 'Neighborhood'].map(mode)

# And for GarageYrBlt, the only numerical variable, we will replace it with the median of the neighborhood from the training set
GarageYrBlt_median_Neighborhood = traindf.groupby('Neighborhood')['GarageYrBlt'].median()
row_label = testdf[testdf["GarageYrBlt"].isnull()].index[0]
neighborhood = testdf.loc[row_label, 'Neighborhood']
testdf.loc[row_label, 'GarageYrBlt'] = GarageYrBlt_median_Neighborhood[neighborhood]

Let's do a final check to make sure all missing values have been dealt with:

In [ ]:
testdf.isnull().sum().sort_values(ascending=False)[lambda x: x > 0]

And now lets save the cleaned test dataset to a file.

In [ ]:
test_clean_output_path = '../data/processed/test_clean.csv'  # Output file path
testdf.to_csv(test_clean_output_path, index=False)

### 2.4 Feature analysis

We will first check how many numerical and categorical features we have.  


In [ ]:
print(f"Categorical columns:\n{traindf.select_dtypes(include=['object']).columns}\n")
print(f"Numerical columns:\n{traindf.select_dtypes(include=['int64', 'float64']).columns}")

Taking a look at this, we can see that we cannot directly separate them by type, as there are some categorical features stored as numbers.  
We are going to manually check the features information on **data_description.txt** and adjust these lists.

In [ ]:
cat_features = [
    'MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
    'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
    'HouseStyle', 'OverallQual', 'OverallCond', 'RoofStyle', 'RoofMatl', 'Exterior1st',
    'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
    'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC',
    'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType',
    'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence',
    'MiscFeature', 'MoSold', 'SaleType', 'SaleCondition'
]

num_features = [
    'LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
    'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
    'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
    'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars',
    'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
    'ScreenPorch', 'PoolArea', 'MiscVal', 'YrSold'
]

In [ ]:
print(f"Number of categorical features: {len(cat_features)}")
print(f"Number of numerical features: {len(num_features)}")

#### 2.4.1 Numerical variables

In [ ]:
for col in num_features:
    print(f"{col}:")
    # Histogram with KDE
    plt.figure(figsize=(8, 5))
    sns.histplot(traindf[col], kde=True)
    plt.title(f'{col} distribution')
    plt.xlabel(col)
    plt.ylabel('Value count')
    plt.show()
    # Boxplot
    plt.figure(figsize=(8, 1))
    sns.boxplot(x=traindf[col])
    plt.title(f'Boxplot of {col}')
    plt.show()
    # Scatter plot against SalePrice
    sns.scatterplot(data=traindf, x=col, y=traindf['SalePrice'])
    plt.title(f"{col} vs. SalePrice")
    plt.show()

#### 2.4.2 Categorical variables

In [ ]:
for col in cat_features:
    print(f"{col}:")
    vc = traindf[col].value_counts()
    print(f'- {len(vc)} unique categories')
    # Count plot
    plt.figure(figsize=(8, 5))
    sns.countplot(data=traindf, x=col, order=vc.index)
    plt.xticks(rotation=45)
    plt.title(f'Value count per category of {col}')
    plt.xlabel('Categories')
    plt.ylabel('Value count')
    plt.show()
    # Boxplot
    plt.figure(figsize=(8, 5))
    sns.boxplot(x=col, y=traindf['SalePrice'], data=traindf)
    plt.title(f"SalePrice by {col}")
    plt.xticks(rotation=45)
    plt.show()

### 2.5 Feature correlation

In [ ]:
corr = traindf[num_features + ["SalePrice"]].corr()
plt.figure(figsize=(15, 13))
sns.heatmap(corr, cmap="coolwarm", annot=True, fmt=".2f", mask=np.triu(np.ones_like(corr, dtype=bool)))
plt.title("Correlation Matrix")
plt.show()

In [ ]:
top_corr = corr['SalePrice'].drop('SalePrice').abs().sort_values(ascending=False).head(5).index.tolist()
sns.pairplot(traindf[top_corr + ['SalePrice']])
plt.show()

In [ ]:
mi_scores = mutual_info_regression(traindf[num_features], traindf['SalePrice'])
mi_scores = pd.Series(mi_scores, index=num_features).sort_values(ascending=False)

sns.barplot(x=mi_scores.values, y=mi_scores.index)
plt.title("Mutual Information Scores")
plt.show()

## 3. Feature engineering  

### 3.1 New features

Most of the models we’ll be using (such as tree-based and deep learning models) are capable of learning interactions between features. However, explicitly providing these interactions through feature engineering can improve learning efficiency, reduce the risk of important relationships being missed, and often lead to better performance with less data.  

With this in mind, we will manually create new features based on available data that capture potentially useful relationships relevant to the task.

We are going to create the following on both sets of data:
- **TotalBathrooms**: Including basement ones
- **TotalSF**: Both floors plus basement
- **FinishedSF**: Total livable area (excluding unfinished basement)
- **Has2ndFloor**: Yes/No
- **HasBasement**: Yes/No
- **HasGarage**: Yes/No
- **HasPool**: Yes/No
- **HouseAge**: Years from when it was build till sale
- **GarageAge**: Years from when it was build till sale
- **RemodelAge**: Years from last remodelation till sale
- **WasRemodeled**: Yes/No
- **QualityIndex**: Ratio expressing overall quality and condition
- **LotRatio**: Ratio expressing relative house to land size (area)

In [ ]:
traindf["TotalBathrooms"] = traindf["FullBath"] + (0.5 * traindf["HalfBath"]) + traindf["BsmtFullBath"] + (0.5 * traindf["BsmtHalfBath"])
testdf["TotalBathrooms"] = testdf["FullBath"] + (0.5 * testdf["HalfBath"]) + testdf["BsmtFullBath"] + (0.5 * testdf["BsmtHalfBath"])

traindf["TotalSF"] = traindf["TotalBsmtSF"] + traindf["1stFlrSF"] + traindf["2ndFlrSF"]
testdf["TotalSF"] = testdf["TotalBsmtSF"] + testdf["1stFlrSF"] + testdf["2ndFlrSF"]

traindf["FinishedSF"] = traindf["BsmtFinSF1"] + traindf["BsmtFinSF2"] + traindf["1stFlrSF"] + traindf["2ndFlrSF"]
testdf["FinishedSF"] = testdf["BsmtFinSF1"] + testdf["BsmtFinSF2"] + testdf["1stFlrSF"] + testdf["2ndFlrSF"]

traindf["Has2ndFloor"] = (traindf["2ndFlrSF"] > 0).astype(int)
testdf["Has2ndFloor"] = (testdf["2ndFlrSF"] > 0).astype(int)

traindf["HasBasement"] = (traindf["TotalBsmtSF"] > 0).astype(int)
testdf["HasBasement"] = (testdf["TotalBsmtSF"] > 0).astype(int)

traindf["HasGarage"] = (traindf["GarageArea"] > 0).astype(int)
testdf["HasGarage"] = (testdf["GarageArea"] > 0).astype(int)

traindf["HasPool"] = (traindf["PoolArea"] > 0).astype(int)
testdf["HasPool"] = (testdf["PoolArea"] > 0).astype(int)

traindf["HouseAge"] = traindf["YrSold"] - traindf["YearBuilt"]
testdf["HouseAge"] = testdf["YrSold"] - testdf["YearBuilt"]

traindf["GarageAge"] = traindf["YrSold"] - traindf["GarageYrBlt"]
testdf["GarageAge"] = testdf["YrSold"] - testdf["GarageYrBlt"]

traindf["RemodelAge"] = traindf["YrSold"] - traindf["YearRemodAdd"]
testdf["RemodelAge"] = testdf["YrSold"] - testdf["YearRemodAdd"]

traindf["WasRemodel"] = (traindf["YearRemodAdd"] != traindf["YearBuilt"]).astype(int)
testdf["WasRemodel"] = (testdf["YearRemodAdd"] != testdf["YearBuilt"]).astype(int)

traindf["QualityIndex"] = traindf["OverallQual"] * traindf["OverallCond"]
testdf["QualityIndex"] = testdf["OverallQual"] * testdf["OverallCond"]

traindf["LotRatio"] = traindf["GrLivArea"] / traindf["LotArea"]
testdf["LotRatio"] = testdf["GrLivArea"] / testdf["LotArea"]

### 3.2 Domain-driven transformations

### 3.3 Drop unneeded columns

Besides this, we will drop the **Id** column in both datasets, as it offers no useful information for this task.

In [ ]:
traindf.drop('Id', axis=1, inplace=True)
testdf.drop('Id', axis=1, inplace=True)

And we will save both engineered datasets.

In [ ]:
train_eng_output_path = '../data/processed/train_engineered.csv'  # Output file path
traindf.to_csv(train_eng_output_path, index=False)

test_eng_output_path = '../data/processed/test_engineered.csv'  # Output file path
testdf.to_csv(test_eng_output_path, index=False)

## 4. Preprocessing

We are working with tabular data that includes both numerical and categorical features.  
Depending on the model type, specific preprocessing (like encoding categorical variables or normalizing numerical features) may be necessary.  

Since each model handles input data differently (some may not benefit from certain transformations or may even be incompatible with them), we will normaliza and/or encode variables when needed, separately for each model.

However, to ensure a fair comparison, we will apply some common transformations beforehand to create our shared "starting point".

### 4.1 Log-transform target variable

Let's transform our target variable **SalePrice**.

In [ ]:
traindf["SalePrice"] = np.log1p(traindf["SalePrice"]).astype(np.float32)

### 4.2 Sanitize variables data types

To avoid mixed data types problems, specially with the neural networks models, we will sanitize the data by making sure all data has the following format:  
- Numerical variables: float32
- Categorical variables: String

First we will update our lists containing which features are numerical and which categorical, to include the changes made on the **Feature engineering** section.

In [ ]:
cat_features = [
    'MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
    'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
    'HouseStyle', 'OverallQual', 'OverallCond', 'RoofStyle', 'RoofMatl', 'Exterior1st',
    'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
    'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC',
    'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType',
    'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence',
    'MiscFeature', 'MoSold', 'SaleType', 'SaleCondition', 'Has2ndFloor', 'HasBasement',
    'HasGarage', 'HasPool', 'WasRemodel'
]

num_features = [
    'LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
    'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
    'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
    'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars',
    'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
    'ScreenPorch', 'PoolArea', 'MiscVal', 'YrSold', 'TotalBathrooms', 'TotalSF',
    'FinishedSF', 'HouseAge', 'GarageAge', 'RemodelAge', 'QualityIndex', 'LotRatio'
]

In [ ]:
print(f"Number of categorical features: {len(cat_features)}")
print(f"Number of numerical features: {len(num_features)}")

And now we will loop over the features changing types where necessary.

In [ ]:
for cat in cat_features:
    traindf[cat] = traindf[cat].astype(str)
    testdf[cat] = testdf[cat].astype(str)

for num in num_features:
    traindf[num] = traindf[num].astype(np.float32)
    testdf[num] = testdf[num].astype(np.float32)

### 4.3 Train/dev/holdout data split

Our dataset is relatively small. As mentioned earlier, while a "test" set has been provided, it does not include the target variable or results.  
This test set will be used only as unseen data for making final predictions—it will not be used for training or evaluation during model development.  

Given this, we need to split our available training data. With approximately 1,500 examples, we will use a 70/20/10 split:

- 70% for training (**train set**)
- 20% for evaluation and fine-tuning during development (**dev set**)
- 10% for final evaluation to assess overfitting and compare models (**holdout set**)

In [ ]:
Y = traindf["SalePrice"]
X = traindf.drop("SalePrice", axis=1)

In [ ]:
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.3, random_state=33)
X_dev, X_hold, Y_dev, Y_hold = train_test_split(X_temp, Y_temp, test_size=1/3, random_state=33)

## 5. Modeling

### 5.1 Baseline model (Ridge regressor)

We will use Scikit-Learn to build a simple linear model.  

In this case we will use Ridge Regression instead of linear regression due to the high number of features (the regularization could help avoid overfitting).

First we will define the preprocessor that will encode the categorical variables using one-hot encoding.  
The numerical variables will not be scaled or normalized as it is not necessary for this model.  

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features),
        # numerical columns are passed through unchanged
        ('num', 'passthrough', num_features)
    ]
)

We will build the model, and fit it to our training data.  
Because this is just a baseline model used for comparison, we will leave the alpha parameter as its default value (1.0), and will not try to improve it any further.

In [ ]:
Ridge_model = Pipeline([
    ("preprocess", preprocessor),
    ("model", Ridge(alpha=1.0))
])

In [ ]:
# Fit the model
Ridge_model.fit(X_train, Y_train)

Now lets evaluate the model on our dev set:

In [ ]:
# Predict on dev set
Y_pred = Ridge_model.predict(X_dev)

In [ ]:
Ridge_dev_rmse_log = root_mean_squared_error(Y_dev, Y_pred)
Ridge_dev_rmse_dollars = root_mean_squared_error(np.expm1(Y_dev), np.expm1(Y_pred))
print(f"Ridge RMSE (log scale): {Ridge_dev_rmse_log:.4f}")
print(f"Random Forest RMSE (dollars): {Ridge_dev_rmse_dollars:.4f}")

And also on our holdout set for comparison purposes:

In [ ]:
# Predict on hold set
Y_pred = Ridge_model.predict(X_hold)

In [ ]:
Ridge_hold_rmse_log = root_mean_squared_error(Y_hold, Y_pred)
Ridge_hold_rmse_dollars = root_mean_squared_error(np.expm1(Y_hold), np.expm1(Y_pred))
print(f"Ridge RMSE (log scale): {Ridge_hold_rmse_log:.4f}")
print(f"Ridge RMSE (dollars): {Ridge_hold_rmse_dollars:.4f}")

Finally, we will store the results for the model comparison later on, and save our final model.

In [ ]:
model_summary = []

model_summary.append({
    'model': 'Ridge Regression',
    'dev_rmse_log': Ridge_dev_rmse_log,
    'dev_rmse_dollars': Ridge_dev_rmse_dollars,
    'holdout_rmse_log': Ridge_hold_rmse_log,
    'holdout_rmse_dollars': Ridge_hold_rmse_dollars
})

In [ ]:
joblib.dump(Ridge_model, '../models/ridge_model.pkl')

del Ridge_model

# Ridge_model = joblib.load('../models/ridge_model.pkl')

### 5.2 Random forest model

We will use TensorFLow Decision Forests to build our Random Forest model.  

TFDF can handle internally the encoding of categorical features, and as such we will be using the engineered (and previously cleaned) datasets directly with no preprocessing.

We will begin with a basic Random Forest model with reasonable hyperparams:

In [ ]:
RF_model = tfdf.keras.RandomForestModel(
    task=tfdf.keras.Task.REGRESSION,        # Define the task (Regression)
    num_trees=300,                          # Number of trees in the forest
    max_depth=10,                           # Maximum depth of trees
    min_examples=5,                         # Minimum number of examples per leaf node
    categorical_algorithm="CART",           # Algorithm for handling categorical features
    compute_oob_variable_importances=True,  # Compute out-of-bag variable importances
)

In [ ]:
RF_model.fit(tfdf.keras.pd_dataframe_to_tf_dataset(
    pd.concat([X_train, Y_train], axis=1), 
    task=tfdf.keras.Task.REGRESSION, label="SalePrice"))

Lets check the out-of-bag performance metrics (train set).

In [ ]:
RF_inspector = RF_model.make_inspector()
print(RF_inspector.evaluation())

And finally lets make the predictions with the dev set and evaluate how our model performed.

In [ ]:
Y_pred = RF_model.predict(tfdf.keras.pd_dataframe_to_tf_dataset(X_dev))

In [ ]:
RF_dev_rmse_log = root_mean_squared_error(Y_dev, Y_pred)
RF_dev_rmse_dollars = root_mean_squared_error(np.expm1(Y_dev), np.expm1(Y_pred))

print(f"Random Forest RMSE (log scale): {RF_dev_rmse_log:.4f}")
print(f"Random Forest RMSE (dollars): {RF_dev_rmse_dollars:.4f}")

We can see already a substantial improvement over our baseline model in the dev set:  
- Ridge Holdout RMSE (log scale): 0.1641
- RandomForest Holdout RMSE (log scale): 0.1383  

But before evaluating on the holdout set, lets finetune our Random Forest model and see how much can it be improved.

In [ ]:
del RF_model # Free up memory

In [ ]:
# Function to evaluate the Random Forest model with different hyperparameters

def evaluate_rf_model(num_trees, max_depth, min_examples):
    RF_model = tfdf.keras.RandomForestModel(
        task=tfdf.keras.Task.REGRESSION,             # Define the task (Regression)
        num_trees=num_trees,                         # Number of trees in the forest
        max_depth=max_depth,                         # Maximum depth of trees
        min_examples=min_examples,                   # Minimum number of examples per leaf node
        categorical_algorithm="CART",                # Algorithm for handling categorical features
        compute_oob_variable_importances=False,      # Compute out-of-bag variable importances
    )

    # Fit the model to the training data
    RF_model.fit(tfdf.keras.pd_dataframe_to_tf_dataset(
        pd.concat([X_train, Y_train], axis=1), 
        task=tfdf.keras.Task.REGRESSION, label="SalePrice"), verbose=0)

    # Predict on dev set and calculate RMSE
    Y_pred = RF_model.predict(tfdf.keras.pd_dataframe_to_tf_dataset(X_dev))
    rmse = root_mean_squared_error(Y_dev, Y_pred)

    # Clean up the model to free memory
    del RF_model

    return rmse

In [ ]:
# Create a list to store the results
RF_tuning = []

# Loop through different hyperparameters
for trees in [100, 300, 500]:
    for depth in [8, 10, 12]:
        for minex in [2, 5, 10]:
            # Evaluate model and get RMSE
            rmse = evaluate_rf_model(trees, depth, minex)
            
            # Append the results to the list
            RF_tuning.append({
                "num_trees": trees,
                "max_depth": depth,
                "min_examples": minex,
                "rmse": rmse
            })

# Convert the results to a DataFrame and sort by RMSE
RF_tuning_df = pd.DataFrame(RF_tuning)
RF_tuning_df = RF_tuning_df.sort_values(by="rmse")

In [ ]:
# Display the results
print(RF_tuning_df)

Lets now fit our final Random Forest model with the best performing set of hyperparameters:  
- num_trees = 500
- max_depth = 12
- min_examples = 5

In [ ]:
RF_model = tfdf.keras.RandomForestModel(
    task=tfdf.keras.Task.REGRESSION,        # Define the task (Regression)
    num_trees=500,                          # Number of trees in the forest
    max_depth=12,                           # Maximum depth of trees
    min_examples=5,                         # Minimum number of examples per leaf node
    categorical_algorithm="CART",           # Algorithm for handling categorical features
    compute_oob_variable_importances=True,  # Compute out-of-bag variable importances
)

In [ ]:
RF_model.fit(tfdf.keras.pd_dataframe_to_tf_dataset(
    pd.concat([X_train, Y_train], axis=1), 
    task=tfdf.keras.Task.REGRESSION, label="SalePrice"))

Lets check the performance on the training set:

In [ ]:
RF_inspector = RF_model.make_inspector()
print(RF_inspector.evaluation())

And evaluate on both the dev and the holdout sets:

In [ ]:
Y_pred = RF_model.predict(tfdf.keras.pd_dataframe_to_tf_dataset(X_dev))

In [ ]:
RF_dev_rmse_log = root_mean_squared_error(Y_dev, Y_pred)
RF_dev_rmse_dollars = root_mean_squared_error(np.expm1(Y_dev), np.expm1(Y_pred))

print(f"Random Forest RMSE (log scale): {RF_dev_rmse_log:.4f}")
print(f"Random Forest RMSE (dollars): {RF_dev_rmse_dollars:.4f}")

In [ ]:
Y_pred = RF_model.predict(tfdf.keras.pd_dataframe_to_tf_dataset(X_hold))

In [ ]:
RF_hold_rmse_log = root_mean_squared_error(Y_hold, Y_pred)
RF_hold_rmse_dollars = root_mean_squared_error(np.expm1(Y_hold), np.expm1(Y_pred))

print(f"Random Forest RMSE (log scale): {RF_hold_rmse_log:.4f}")
print(f"Random Forest RMSE (dollars): {RF_hold_rmse_dollars:.4f}")

Performance has not worsened on the holdout set (it got even better but that is probably just due to the small holdout sample), which means weare probably not overfitting the data.  

As the final step, we will store the performance metrics for the final comparison, and save our model so we can load it up again anytime without wasting time training it.

In [ ]:
model_summary.append({
    'model': 'Random Forest',
    'dev_rmse_log': RF_dev_rmse_log,
    'dev_rmse_dollars': RF_dev_rmse_dollars,
    'holdout_rmse_log': RF_hold_rmse_log,
    'holdout_rmse_dollars': RF_hold_rmse_dollars
})

In [ ]:
RF_model.save("../models/rf_model")	

del RF_model

# Load the model
# RF_model = tfdf.keras.models.load_model("../models/rf_model")

### 5.3 Catboost model

The next model we are going to train is going to be a gradient boosting model, specifically the regressor model of the CatBoost framework.  

Gradient boosting models work exceptionally well with tabular data, like other tree-based models, and for a dataset such as this one, small in size and with a relatively big number of categorical features, CatBoost models outperform most models even without much finetuning.  

Given the type of task and data, this is probably going to be our best performing model, but let's not jump ahead of ourselves and proceed as usual, and we will see what happens by the end.

Before training any models, we need to transform our data to the appropiate format native to this framework:

In [ ]:
# Create Pools (CatBoost's data structure)
train_pool = Pool(data=X_train, label=Y_train, cat_features=cat_features)
dev_pool = Pool(data=X_dev, label=Y_dev, cat_features=cat_features)
hold_pool = Pool(data=X_hold, label=Y_hold, cat_features=cat_features)

Now we will train a basic model with reasonable hyperparams to check that everything works before trying to finetune it

In [ ]:
# Define and create the model
def create_CB_model(lr, depth, l2reg, bag_temp, random_seed=33, verbose=100):
    CB_model = CatBoostRegressor(
        iterations=3000,
        early_stopping_rounds=50,
        learning_rate=lr,
        depth=depth,
        l2_leaf_reg=l2reg,
        bagging_temperature=bag_temp,
        eval_metric='RMSE',
        random_seed=random_seed,
        verbose=verbose  # Print progress after how many iterations
    )
    return CB_model

In [ ]:
# Create basic model
CB_model = create_CB_model(0.1, 6, 3, 1.0)

In [ ]:
# Fit the model
CB_model.fit(train_pool, eval_set=dev_pool, use_best_model=True)

And now we can check the RMSE obtained with the dev set:

In [ ]:
CB_model.get_best_score()['validation']['RMSE']

We can already see an improvement against our baseline, but with worse performance than our final Random Forest model.  
But just as we did withe the latter, we will finetune its hyperparams to try to boost its performance.

First we will create the list of parameters to iterate over.  
Note that we wont be trying different numbers of iterations, because we set a farly big number of iterations (3000) with early stopping, so it will automatically stop if and when it plateaus or start overfitting.

In [ ]:
del CB_model # Delete the basic model we just created

lr_values = [0.01, 0.03, 0.1, 0.2]
depth_values = [4, 6, 8, 10]
L2_values = [1, 3, 5, 10]
bag_temp_values = [0, 0.5, 1.0, 5.0]

And then we will use a nested loop to try all the combinations while storing the performance metrics for comparison:

In [ ]:
catboost_results = []

for lr in lr_values:
    for depth in depth_values:
        for l2reg in L2_values:
            for bag_temp in bag_temp_values:
                # Define and train model
                CB_model = create_CB_model(lr, depth, l2reg, bag_temp, verbose=0)
                
                # Fit the model
                CB_model.fit(train_pool, eval_set=dev_pool, use_best_model=True)

                # Get rmse value
                rmse = CB_model.get_best_score()['validation']['RMSE']

                # Store results
                catboost_results.append({
                    "learning_rate": lr,
                    "depth": depth,
                    "L2_leaf_reg": l2reg,
                    "bagging_temperature": bag_temp,
                    "rmse": rmse})
                
                # Clean up memory
                del CB_model

# Convert to DataFrame
catboost_results_df = pd.DataFrame(catboost_results)

# Sort by best RMSE
catboost_results_df = catboost_results_df.sort_values(by="rmse")

# Show best settings:
print("\nTop 10 settings by RMSE:")
catboost_results_df.head(10)

Given this, our chosen hyperparams are the following:  
- learning_rate = 0.03
- depth = 6
- L2_leaf_reg = 1.0
- bagging_temperature = 1.0 (default, as results showed no change)

Now we will train this with different random_seeds to check for stability, to ensure that our performance is not due to a lucky match.

In [ ]:
random_seeds = [11, 33, 42, 55, 64]

for seed in random_seeds:
    # Define and train model
    CB_model = create_CB_model(0.03, 6, 1, 1.0, random_seed=seed, verbose=0)

    # Fit the model
    CB_model.fit(train_pool, eval_set=dev_pool, use_best_model=True)

    # Get rmse value
    rmse = CB_model.get_best_score()['validation']['RMSE']

    print(f"Random seed {seed}: RMSE = {rmse:.4f}")
    # Clean up
    del CB_model

The performance is roughly similar, which means we can proceed forward with building our final CatBoost model.

In [ ]:
# Create final model
CB_model = create_CB_model(0.03, 6, 1, 1.0, verbose=100)

# Fit the model
CB_model.fit(train_pool, eval_set=dev_pool, use_best_model=True)

And now we can evaluate the final performance on both dev and holdout sets:

In [ ]:
Y_pred = CB_model.predict(dev_pool)

In [ ]:
CB_dev_rmse_log = root_mean_squared_error(Y_dev, Y_pred)
CB_dev_rmse_dollars = root_mean_squared_error(np.expm1(Y_dev), np.expm1(Y_pred))
print(f"CatBoost RMSE (log scale): {CB_dev_rmse_log:.4f}")
print(f"CatBoost RMSE (dollars): {CB_dev_rmse_dollars:.4f}")

In [ ]:
Y_pred = CB_model.predict(hold_pool)

In [ ]:
CB_hold_rmse_log = root_mean_squared_error(Y_hold, Y_pred)
CB_hold_rmse_dollars = root_mean_squared_error(np.expm1(Y_hold), np.expm1(Y_pred))
print(f"CatBoost RMSE (log scale): {CB_hold_rmse_log:.4f}")
print(f"CatBoost RMSE (dollars): {CB_hold_rmse_dollars:.4f}")

And finally, we will store the results and export our model:

In [ ]:
model_summary.append({
    'model': 'CatBoost model',
    'dev_rmse_log': CB_dev_rmse_log,
    'dev_rmse_dollars': CB_dev_rmse_dollars,
    'holdout_rmse_log': CB_hold_rmse_log,
    'holdout_rmse_dollars': CB_hold_rmse_dollars
})

In [ ]:
CB_model.save_model("../models/catboost_model.cbm")

del CB_model

# Load the model
# CB_model = CatBoostRegressor()
# CB_model.load_model("../models/catboost_model.cbm")

### 5.4 MLP model

For our next model, we will use a neural network model, with a basic MLP architecture.

First the model will normalize all numerical features, and it will encode the categorical ones.  
Given the amount of categorical variables and their cardinality (most under 10, but a few of them go between 15 and 20, with **Neighborhood** having 25 categories), maybe embedding would be a better option than one-hot encoding, but given that the cardinality in general is not that high, and that this model is probably going to be outperformed by others, we will keep it simpler with one-hot.  


After preprocessing the input it will then use 2 fully connected layers, with 128 and 64 hidden units, and with 'Relu' as activation function.  

As output layer given that this is a regression task, it will have a fully connected layer with one unit and no activation.

In [ ]:
def MLP_instance(num_features, cat_features, X_train):
    # Input layer
    # This will create a dictionary which contains an input layer per feature, to help us preprocess them individually
    inputs = {}
    for name in num_features + cat_features:
        inputs[name] = Input(shape=(1,), name=name, dtype='float32' if name in num_features else 'string')

    # Preprocessing for numerical
    norm_layers = {}
    for feature in num_features:
        norm = Normalization() # Create a Normalization layer
        norm.adapt(X_train[feature].values.reshape(-1, 1))  # Gets the mean and variance of the training data, reshape needed to transform (n_samples,) to (n_samples, 1)
        norm_layers[feature] = norm(inputs[feature]) # Apply normalization to the input feature

    # Preprocessing for categorical
    cat_layers = {}
    for feature in cat_features:
        lookup = StringLookup(output_mode='int') # Layer to map strings to integers
        lookup.adapt(X_train[feature].values) # Learns all unique values (categories) of the feature to map them to integers
        
        vocab_size = lookup.vocabulary_size() # Number of unique values (categories) in the feature + 1 for the UNK token
        encoding = CategoryEncoding(output_mode='one_hot', num_tokens=vocab_size) # Layer to convert the integer encoded feature to one-hot encoding
        
        int_encoded = lookup(inputs[feature]) # Apply the lookup layer to the features
        one_hot_encoded = encoding(int_encoded) # Apply the encoding layer to the integer encoded feature
        cat_layers[feature] = one_hot_encoded # Store the one-hot encoded tensor to the dict

    # Combine all features
    all_features = list(norm_layers.values()) + list(cat_layers.values())
    x = Concatenate()(all_features)

    # MLP
    x = Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(x)
    x = Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(x)
    output = Dense(1)(x)

    model = Model(inputs=inputs, outputs=output)

    return model

We will train the model once to make sure everything is working, and for that first of all we need to create the basic model.

In [ ]:
MLP_model = MLP_instance(num_features, cat_features, X_train)

Now let's configure it to use Adam as optimization algorithm, using MSE as loss function and we will use RMSE as metric.

In [ ]:
MLP_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse', metrics=[RootMeanSquaredError()])

We need to transform our training data to the format TF expects (dict of column name - array of values).

In [ ]:
X_train_inputs = {
    col: X_train[col].values for col in num_features + cat_features
}

And finally lets train the model:

In [ ]:
MLP_model.fit(X_train_inputs, Y_train.values, epochs=20, batch_size=32)

We will apply the same transformation to the dev set for the evaluation:

In [ ]:
X_dev_inputs = {
    col: X_dev[col].values for col in num_features + cat_features
}

In [ ]:
loss, rmse = MLP_model.evaluate(X_dev_inputs, Y_dev.values)
print(f"MLP RMSE (log scale): {rmse:.4f}, loss: {loss:.4f}")

The model is working fine but with poor performance compared to even our baseline model.  

But for comparison purposes, we will finetune it to see how much of an improvement we can get and to confirm whether or not this model can offer competitive results compared with the others.  

Given the small dataset instead of using doing cross validation with k-fold, which would reduce even more the training data, we will keep it simpler with a manual grid search, looping through different random seeds to reduce variance on the results.  

We will repeat the process (build-train-evaluate) for different sets of values (number of epochs, batch sizes and learning rates) and we will do it with different seeds averaging the performance metrics obtained (mean of RMSE and its standard deviation). This is necessary to improve generalization and ensure we dont get a good result just due to luck with a lucky initial state (random seed).  

In [ ]:
epochs_list = [20, 100]
batch_sizes = [16, 32, 64]
learning_rates = [1e-3, 1e-2]
random_seeds = [11, 33, 42, 55, 64]

In [ ]:
# Callbacks for early stopping and learning rate scheduler
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=7, restore_best_weights=True
)
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6
)

In [ ]:
MLP_results = [] # Create an empty list to store results

# Grid search loop
for lr in learning_rates:
    for bs in batch_sizes:
        for ep in epochs_list:
            val_rmse_list = [] # List to store RMSE for each random seed
            epochs_trained_list = [] # List to store number of epochs trained for each random seed
            for seed in random_seeds:
                
                tf.keras.utils.set_random_seed(seed)

                # New model instance
                MLP_model = MLP_instance(num_features, cat_features, X_train)

                # Compile model
                MLP_model.compile(
                    optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                    loss='mse',
                    metrics=[RootMeanSquaredError()]
                )

                # Fit the model
                trained_model = MLP_model.fit(
                    X_train_inputs, Y_train.values,
                    validation_data=(X_dev_inputs, Y_dev.values),
                    epochs=ep,
                    batch_size=bs,
                    callbacks=[early_stop, lr_scheduler],
                    verbose=0
                )

                # Evaluate
                loss, rmse = MLP_model.evaluate(X_dev_inputs, Y_dev.values, verbose=0)

                val_rmse_list.append(rmse) # Append RMSE to the list
                epochs_trained_list.append(len(trained_model.history['loss'])) # Append number of epochs trained to the list

                del MLP_model # Clear the model from memory
                tf.keras.backend.clear_session() # Clear the session to free up resources

            # Store result in a dict and append to the results list
            MLP_results.append({
                "learning_rate": lr,
                "batch_size": bs,
                "epochs_requested": ep,
                "epochs_trained_mean": np.mean(epochs_trained_list),
                "val_rmse_mean": np.mean(val_rmse_list),
                "val_rmse_std": np.std(val_rmse_list),
            })

# Convert to DataFrame
MLP_results_df = pd.DataFrame(MLP_results)

# Sort by best RMSE
MLP_results_df = MLP_results_df.sort_values(by="val_rmse_mean")

# Show best settings:
print("\nTop 10 settings by RMSE:")
MLP_results_df.head(10)

We will go forward with the best option and train again the model for a final evaluation on the holdout set, for later comparison with the other models.

Model building and training:

In [ ]:
MLP_model = MLP_instance(num_features, cat_features, X_train)
MLP_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss='mse', metrics=[RootMeanSquaredError()])
MLP_model.fit(X_train_inputs, Y_train.values, validation_data=(X_dev_inputs, Y_dev.values), epochs=100, batch_size=16, callbacks=[early_stop, lr_scheduler])

Model evaluation on the dev set:

In [ ]:
Y_pred = MLP_model.predict(X_dev_inputs)

In [ ]:
MLP_dev_rmse_log = root_mean_squared_error(Y_dev, Y_pred)
MLP_dev_rmse_dollars = root_mean_squared_error(np.expm1(Y_dev), np.expm1(Y_pred))
print(f"MLP RMSE (log scale): {MLP_dev_rmse_log:.4f}")
print(f"MLP RMSE (dollars): {MLP_dev_rmse_dollars:.4f}")

And finally, we will transform the holdout set just as we did with the train and dev sets, and perform the last evaluation on it:

In [ ]:
X_hold_inputs = {
    col: X_hold[col].values for col in num_features + cat_features
}

In [ ]:
Y_pred = MLP_model.predict(X_hold_inputs)

In [ ]:
MLP_hold_rmse_log = root_mean_squared_error(Y_hold, Y_pred)
MLP_hold_rmse_dollars = root_mean_squared_error(np.expm1(Y_hold), np.expm1(Y_pred))
print(f"MLP RMSE (log scale): {MLP_hold_rmse_log:.4f}")
print(f"MLP RMSE (dollars): {MLP_hold_rmse_dollars:.4f}")

Finally, lets store our performance results and save our MLP model:

In [ ]:
model_summary.append({
    'model': 'MLP model',
    'dev_rmse_log': MLP_dev_rmse_log,
    'dev_rmse_dollars': MLP_dev_rmse_dollars,
    'holdout_rmse_log': MLP_hold_rmse_log,
    'holdout_rmse_dollars': MLP_hold_rmse_dollars
})

In [ ]:
MLP_model.save("../models/mlp_model.keras")

del MLP_model
tf.keras.backend.clear_session()

# Load the model
# MLP_model = tf.keras.models.load_model("../models/mlp_model.keras", custom_objects={'RootMeanSquaredError': RootMeanSquaredError})

### 5.5 Multi Level Dense Layer NN model

Multi Level Dense Layer Neural Network

In [ ]:
def MLDL_instance(num_features, cat_features, X_train):
    # Input layer
    # This will create a dictionary which contains an input layer per feature, to help us preprocess them individually
    inputs = {}
    for name in num_features + cat_features:
        inputs[name] = Input(shape=(1,), name=name, dtype='float32' if name in num_features else 'string')

    # Preprocessing for numerical
    norm_layers = {}
    for feature in num_features:
        norm = Normalization() # Create a Normalization layer
        norm.adapt(X_train[feature].values.reshape(-1, 1))  # Gets the mean and variance of the training data, reshape needed to transform (n_samples,) to (n_samples, 1)
        norm_layers[feature] = norm(inputs[feature]) # Apply normalization to the input feature

    # Preprocessing for categorical
    cat_layers = {}
    for feature in cat_features:
        lookup = StringLookup(output_mode='int') # Layer to map strings to integers
        lookup.adapt(X_train[feature].values) # Learns all unique values (categories) of the feature to map them to integers
        
        vocab_size = lookup.vocabulary_size() # Number of unique values (categories) in the feature + 1 for the UNK token
        encoding = CategoryEncoding(output_mode='one_hot', num_tokens=vocab_size) # Layer to convert the integer encoded feature to one-hot encoding
        
        int_encoded = lookup(inputs[feature]) # Apply the lookup layer to the features
        one_hot_encoded = encoding(int_encoded) # Apply the encoding layer to the integer encoded feature
        cat_layers[feature] = one_hot_encoded # Store the one-hot encoded tensor to the dict

    # Combine all features
    all_features = list(norm_layers.values()) + list(cat_layers.values())
    x = Concatenate()(all_features)

    # Split into 3 branches for multi-level dense layers
    branch_1 = Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(x)
    branch_1 = Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(branch_1)
    branch_1 = Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(branch_1)

    branch_2 = Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(x)

    branch_3 = Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(x)

    # Merge branches
    merged = Concatenate()([branch_1, branch_2, branch_3])
    merged = Dense(8, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(merged)
    output = Dense(1)(merged)

    model = Model(inputs=inputs, outputs=output)

    return model

In [ ]:
# Callbacks for early stopping and learning rate scheduler
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=7, restore_best_weights=True
)
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6
)

In [ ]:
MLDL_model = MLDL_instance(num_features, cat_features, X_train)
MLDL_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
                     loss='mse',
                     metrics=[RootMeanSquaredError()])

MLDL_model.fit(X_train_inputs, Y_train.values,
                 validation_data=(X_dev_inputs, Y_dev.values),
                 epochs=100,
                 batch_size=16,
                 callbacks=[early_stop, lr_scheduler])

In [ ]:
loss, rmse = MLDL_model.evaluate(X_dev_inputs, Y_dev.values)
print(f"Multi Level Dense Layer NN RMSE (log scale): {rmse:.4f}, loss: {loss:.4f}")

In [ ]:
epochs_list = [20, 100]
batch_sizes = [16, 32, 64]
learning_rates = [1e-3, 1e-2]
random_seeds = [11, 33, 42, 55, 64]

In [ ]:
MLDL_results = [] # Create an empty list to store results

# Grid search loop
for lr in learning_rates:
    for bs in batch_sizes:
        for ep in epochs_list:
            val_rmse_list = [] # List to store RMSE for each random seed
            epochs_trained_list = [] # List to store number of epochs trained for each random seed
            for seed in random_seeds:
                
                tf.keras.utils.set_random_seed(seed)

                # New model instance
                MLDL_model = MLDL_instance(num_features, cat_features, X_train)

                # Compile model
                MLDL_model.compile(
                    optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                    loss='mse',
                    metrics=[RootMeanSquaredError()]
                )

                # Fit the model
                trained_model = MLDL_model.fit(
                    X_train_inputs, Y_train.values,
                    validation_data=(X_dev_inputs, Y_dev.values),
                    epochs=ep,
                    batch_size=bs,
                    callbacks=[early_stop, lr_scheduler],
                    verbose=0
                )

                # Evaluate
                loss, rmse = MLDL_model.evaluate(X_dev_inputs, Y_dev.values, verbose=0)

                val_rmse_list.append(rmse) # Append RMSE to the list
                epochs_trained_list.append(len(trained_model.history['loss'])) # Append number of epochs trained to the list

                del MLDL_model # Clear the model from memory
                tf.keras.backend.clear_session() # Clear the session to free up resources

            # Store result in a dict and append to the results list
            MLDL_results.append({
                "learning_rate": lr,
                "batch_size": bs,
                "epochs_requested": ep,
                "epochs_trained_mean": np.mean(epochs_trained_list),
                "val_rmse_mean": np.mean(val_rmse_list),
                "val_rmse_std": np.std(val_rmse_list),
            })

# Convert to DataFrame
MLDL_results_df = pd.DataFrame(MLDL_results)

# Sort by best RMSE
MLDL_results_df = MLDL_results_df.sort_values(by="val_rmse_mean")

# Show best settings:
print("\nTop 10 settings by RMSE:")
MLDL_results_df.head(10)

In [ ]:
del MLDL_model
tf.keras.backend.clear_session()
tf.keras.utils.set_random_seed(33)

In [ ]:
MLDL_model = MLDL_instance(num_features, cat_features, X_train)
MLDL_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss='mse', metrics=[RootMeanSquaredError()])
MLDL_model.fit(X_train_inputs, Y_train.values, validation_data=(X_dev_inputs, Y_dev.values), epochs=150, batch_size=16, callbacks=[early_stop, lr_scheduler])

In [ ]:
Y_pred = MLDL_model.predict(X_dev_inputs)

In [ ]:
MLDL_dev_rmse_log = root_mean_squared_error(Y_dev, Y_pred)
MLDL_dev_rmse_dollars = root_mean_squared_error(np.expm1(Y_dev), np.expm1(Y_pred))
print(f"MLDL RMSE (log scale): {MLDL_dev_rmse_log:.4f}")
print(f"MLDL RMSE (dollars): {MLDL_dev_rmse_dollars:.4f}")

In [ ]:
Y_pred = MLDL_model.predict(X_hold_inputs)

In [ ]:
MLDL_hold_rmse_log = root_mean_squared_error(Y_hold, Y_pred)
MLDL_hold_rmse_dollars = root_mean_squared_error(np.expm1(Y_hold), np.expm1(Y_pred))
print(f"MLDL RMSE (log scale): {MLDL_hold_rmse_log:.4f}")
print(f"MLDL RMSE (dollars): {MLDL_hold_rmse_dollars:.4f}")

In [ ]:
model_summary.append({
    'model': 'MLDL model',
    'dev_rmse_log': MLDL_dev_rmse_log,
    'dev_rmse_dollars': MLDL_dev_rmse_dollars,
    'holdout_rmse_log': MLDL_hold_rmse_log,
    'holdout_rmse_dollars': MLDL_hold_rmse_dollars
})

In [ ]:
MLDL_model.save("../models/mldl_model.keras")

del MLDL_model
tf.keras.backend.clear_session()

# Load the model
# MLDL_model = tf.keras.models.load_model("../models/mldl_model.keras", custom_objects={'RootMeanSquaredError': RootMeanSquaredError})

## 6. Model comparison

We have been storing the metrics of the tuned models, lets order them by performance and see how they compare.

In [ ]:
model_summary_df = pd.DataFrame(model_summary)
model_summary_df = model_summary_df.sort_values(by="holdout_rmse_log") # Sort by best RMSE (log scale) on the holdout set

model_summary_df

As we can see our CatBoost Regressor model outperforms all others, which is to be expected given the size of the dataset and the nature of the data.  

In [ ]:
# Save the model summary to a CSV file
model_summary_df.to_csv('../outputs/comparison_summary.csv', index=False)

# Load the model summary
# model_summary_df = pd.read_csv('../outputs/comparison_summary.csv')

## 7. Final model prediction & submission
### 7.1 Retrain on full training set
### 7.2 Predict on test set
### 7.3 Reverse log-transformation
### 7.4 Generate submission